# 问题列表

## 常用图像标注工具

- [labelimg](https://github.com/tzutalin/labelImg)
- [labelme](https://github.com/wkentaro/labelme)
- [RectLable](https://rectlabel.com)
- [OpenCV/CVAT](https://github.com/cvat)
- [VOTT](https://github.com/microsoft/VoTT)
- [LabelBox](https://github.com/Labelbox/Labelbox)
- [VIA-VGG](http://www.robots.xo.ac.uk/~vgg/software/via)
- [PixelAnnotationTool](https://github.com/abreheret/PixelAnnotationTool)
- [point-cloud-annotation-tool](https://github.com/springzfx/point-clout-annotation-tool)
- [Boobs](https://github.com/drainingsum/boobs)

## labelme 数据标注 转换为VOC格式

```shell
# It generates:
#   - data_dataset_voc/JPEGImages
#   - data_dataset_voc/Annotations
#   - data_dataset_voc/AnnotationsVisualization
./labelme2voc.py data_annotated data_dataset_voc --labels labels.txt
```

## FFmpeg python 视屏抽帧

In [ ]:
import ffmpeg
import numpy as np
import os
import cv2


def get_video_shape(videopath):
    probe = ffmpeg.probe(videopath)
    video_stream = next(
        (stream
         for stream in probe['streams'] if stream['codec_type'] == 'video'),
        None)
    width = int(video_stream['width'])
    height = int(video_stream['height'])

    return width, height


def extract_frames_from_video(videopath, outdir=".", fps=10):
    width, height = get_video_shape(videopath)
    process = ffmpeg.input(videopath).filter(
        "fps", fps=fps,
        round="up").output('pipe:', format='rawvideo',
                           pix_fmt='rgb24').run_async(pipe_stdout=True)

    frame_id = 0
    while True:
        in_bytes = process.stdout.read(width * height * 3)
        if not in_bytes:
            break
        frame = (np.frombuffer(in_bytes, np.uint8).reshape([height, width, 3]))
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        cv2.imwrite(
            os.path.join(outdir,
                         os.path.splitext(os.path.split(videopath)[-1])[0]) +
            f"_{frame_id}.jpg", frame)
        frame_id += 1
    process.wait()


data_dirs = [
    "/opt/data/dataset/smoking/kinetics400smoking",
    "/opt/data/dataset/smoking/hmdb51-smoking/test/smoking",
    "/opt/data/dataset/smoking/hmdb51-smoking/train/smoking"
]

dst_dir = "/opt/data/dataset/smoking/extracts"

videofiles = []
for dir in data_dirs:
    videofiles += [
        os.path.join(dir, filename) for filename in next(os.walk(dir))[-1]
    ]

for videopath in videofiles:
    print(f"extract frame video :  {videopath}")
    extract_frames_from_video(videopath, outdir=dst_dir, fps=0.25)

## FFMPEG 编译安装

1.  配置(其中GPU支持需要安装nv-codec-headers)
```bash
# 默认CPU版本
./configure --enable-shared --disable-ffplay --disable-doc --disable-x86asm
```

`nv-codec-headers`安装
```bash
git clone https://github.com/FFmpeg/nv-codec-headers.git
cd nv-codec-headers
make && make install
```

```bash
# GPU支持
./configure --enable-shared --disable-ffplay --disable-doc --enable-cuda-nvcc --enable-nonfree --disable-x86asm --enable-avresample --enable-cuvid --enable-nvdec --enable-nvenc
```

2. 安装
```bash
make && make install
```

## libSM.so.6

opencv-python 在`import cv2`的时候出现

```bash
Traceback (most recent call last):
  File "main_server.py", line 4, in <module>
    import cv2
  File "/usr/local/lib/python3.6/dist-packages/cv2/__init__.py", line 3, in <module>
    from .cv2 import *
ImportError: libSM.so.6: cannot open shared object file: No such file or directory
```

解决办法：


```bash
apt install libsm-dev
```

##  libXrender.so.1

opencv-python 在`import cv2`的时候出现

```bash
Traceback (most recent call last):
  File "main_server.py", line 4, in <module>
    import cv2
  File "/usr/local/lib/python3.6/dist-packages/cv2/__init__.py", line 3, in <module>
    from .cv2 import *
ImportError: libXrender.so.1: cannot open shared object file: No such file or directory
```

解决办法：

```bash
apt install libxrender-dev
```

## Docker 

1. 删除所有无用镜像：

```bash
sudo docker image rm $(sudo docker images | grep "<none>" | awk '{print $3}')
```

2. 删除所有容器：

```bash
sudo docker rm $(sudo docker ps -aq)
```

3. 提交镜像：

```bash
sudo docker commit 9b91f151eb40 ccr.ccs.tencentyun.com/zyuegege/testimage:mydev
```

## OpenCV 编译(支持CUDA加速)
1. 执行`cmake`
```bash
cd opencv-4.1.2/build
cmake \
       -DCMAKE_BUILD_TYPE=Release \
       -DOPENCV_EXTRA_MODULES_PATH=../../opencv_contrib/modules \
       -DPYTHON3_EXECUTABLE=/usr/bin/python3.6m \
       -DPYTHON_INCLUDE_DIR=/usr/include/python3.6m \
       -DPYTHON_INCLUDE_DIR2=/usr/include/x86_64-linux-gnu/python3.6m \
       -DPYTHON_LIBRARY=/usr/lib/x86_64-linux-gnu/libpython3.6m.so \
       -DOPENCV_GENERATE_PKGCONFIG=ON \
       -DBUILD_opencv_python3=ON \
       -DBUILD_opencv_python2=OFF \
       -DBUILD_JAVA=OFF \
       -DWITH_CUDA=ON \
       -DWITH_FFMPEG=ON  \
       -DPYTHON3_NUMPY_INCLUDE_DIRS=/usr/include/python3.6/numpy
       ..
```

## jupyter lab 


### 问题 `KeyError: 'registry'`:

```bash
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 528, in get
    value = obj._trait_values[self.name]
KeyError: 'registry'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/bin/jupyter-lab", line 5, in <module>
    from jupyterlab.labapp import main
  File "/usr/local/lib/python3.6/dist-packages/jupyterlab/labapp.py", line 40, in <module>
    app_version = get_app_version()
  File "/usr/local/lib/python3.6/dist-packages/jupyterlab/commands.py", line 541, in get_app_version
    app_options, app_dir=app_dir, core_config=core_config))
  File "/usr/local/lib/python3.6/dist-packages/jupyterlab/commands.py", line 582, in __init__
    self.registry = options.registry
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 556, in __get__
    return self.get(obj, cls)
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 535, in get
    value = self._validate(obj, dynamic_default())
  File "/usr/local/lib/python3.6/dist-packages/jupyterlab/commands.py", line 343, in _default_registry
    config = _yarn_config(self.logger)["yarn config"]
  File "/usr/local/lib/python3.6/dist-packages/jupyterlab/commands.py", line 1765, in _yarn_config
    node = which('node')
  File "/usr/local/lib/python3.6/dist-packages/jupyterlab_server/process.py", line 59, in which
    raise ValueError(msg)
ValueError: Please install nodejs 5+ and npm before continuing installation. nodejs may be installed using conda or directly from the nodejs website.
```

解决办法:
```bash
pip install jupyterlab==1.1.3
```